In [1]:
import numpy as np
from data_handler import StaticDataHandler
from model_builder import ModelBuilder
from constants import *
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
# from sklearn.model_selection import train_test_split
data = StaticDataHandler._load_h5py_data(H5PY_PATH)
print(len(data['inputs']))
print(len(data['targets']))
print("Train Test.....")
_train_data, _test_data = StaticDataHandler._train_split_test(split_ratio=0.2, input_data=data)
# X_train, X_test, y_train, y_test = train_test_split(data['inputs'], data['targets'], test_size=0.33, random_state=42)
print("Train Validation.....")
# X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.33, random_state=42)
_train_data, _validation_data = StaticDataHandler._train_split_test(split_ratio=0.2, input_data=_train_data)
mappings = StaticDataHandler._load_jsonfile_into_song(MAPPING_PATH)

2025-01-30 00:52:49.759064: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-30 00:52:50.044435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738169570.159199     699 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738169570.184344     699 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 00:52:50.397428: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Datasets in the file: ['inputs', 'targets']
Inputs shape: (1140090, 64, 45)
Targets shape: (1140090,)
1140090
1140090
Train Test.....
Train Validation.....


In [2]:
print("Data type:", type(mappings))
print("Total length:", len(mappings))
# print("Inputs Train Length", len(X_train))
# print("Inputs Validation Length", len(X_validation))
# print("Inputs Test Length", len(X_test))
# print("Total Length: ", len(X_train) + len(X_validation) + len(X_test))
print("Inputs Train Length", len(_train_data['inputs']))
print("Inputs Validation Length", len(_validation_data['inputs']))
print("Inputs Test Length", len(_test_data['inputs']))
print("Total Length: ", len(_train_data['inputs']) + len(_validation_data['inputs']) + len(_test_data['inputs']))

Data type: <class 'dict'>
Total length: 45
Inputs Train Length 729660
Inputs Validation Length 182413
Inputs Test Length 228017
Total Length:  1140090


In [3]:
# m = ModelBuilder()
# model = m._build_model()
model = load_model('model_checkpoint_epoch_20.keras')
model.summary()

I0000 00:00:1738169598.445944     699 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1738169598.454607     699 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, 45)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, None, 128)      │        89,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, None, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 45)             │         2,925 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 291,548 (1.11 MB)

 Trainable params: 145,709 (569.18 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 145,711 (569.19 KB)

In [4]:
# # Train
# history = model.fit(_train_data['inputs'], _train_data['targets'], 
#                     validation_data=(_validation_data['inputs'], _validation_data['targets']), 
#                     batch_size=BATCH_SIZE, epochs=NUMBER_EPOCH, validation_freq=5)
# # Evaluate
# test_error, test_accuracy = model.evaluate(_test_data['inputs'], _test_data['targets'], verbose=1)
# print("Accuracy on Test Set: ", test_accuracy)
# print("Error on Test Set: ", test_error)


# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((_train_data['inputs'], _train_data['targets']))
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

validation_dataset = tf.data.Dataset.from_tensor_slices((_validation_data['inputs'], _validation_data['targets']))
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((_test_data['inputs'], _test_data['targets']))
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [5]:

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    'model_checkpoint_epoch_{epoch}.keras',  # Save after each epoch
    save_best_only=True,                    # Save the model at every epoch, if False
    save_weights_only=False,                 # Save the entire model (architecture + weights + optimizer state)
    verbose=1
)
# Define learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor='loss',   # Watch the loss for stopping condition
    factor=0.1,       # Reduce learning rate by 10% when the loss plateaus
    patience=3        # Wait for 3 epochs with no improvement
)
# Train
import threading
threading.Thread(target=lambda: model.fit(
                    train_dataset, 
                    validation_data=validation_dataset, 
                    epochs=NUMBER_EPOCH, initial_epoch=21, validation_freq=5, callbacks=[checkpoint_callback, lr_scheduler])).start()

Epoch 22/50


I0000 00:00:1738169653.763267     907 cuda_dnn.cc:529] Loaded cuDNN version 90300


22802/22802 ━━━━━━━━━━━━━━━━━━━━ 481s 21ms/step - loss: 0.5660 - sparse_categorical_accuracy: 0.8353 - learning_rate: 0.0010
Epoch 23/50
    6/22802 ━━━━━━━━━━━━━━━━━━━━ 7:57 21ms/step - loss: 0.4251 - sparse_categorical_accuracy: 0.8741

/home/nigelchua0412/projects/tf217/venv-tf217/lib/python3.12/site-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


22802/22802 ━━━━━━━━━━━━━━━━━━━━ 464s 20ms/step - loss: 0.5656 - sparse_categorical_accuracy: 0.8364 - learning_rate: 0.0010
Epoch 24/50
22802/22802 ━━━━━━━━━━━━━━━━━━━━ 472s 21ms/step - loss: 0.5621 - sparse_categorical_accuracy: 0.8371 - learning_rate: 0.0010
Epoch 25/50
22801/22802 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.5610 - sparse_categorical_accuracy: 0.8378
Epoch 25: val_loss improved from inf to 0.53176, saving model to model_checkpoint_epoch_25.keras
22802/22802 ━━━━━━━━━━━━━━━━━━━━ 543s 24ms/step - loss: 0.5610 - sparse_categorical_accuracy: 0.8378 - val_loss: 0.5318 - val_sparse_categorical_accuracy: 0.8420 - learning_rate: 0.0010
Epoch 26/50
22802/22802 ━━━━━━━━━━━━━━━━━━━━ 608s 27ms/step - loss: 0.5587 - sparse_categorical_accuracy: 0.8382 - learning_rate: 0.0010
Epoch 27/50
22802/22802 ━━━━━━━━━━━━━━━━━━━━ 634s 28ms/step - loss: 0.5568 - sparse_categorical_accuracy: 0.8388 - learning_rate: 0.0010
Epoch 28/50
22802/22802 ━━━━━━━━━━━━━━━━━━━━ 627s 28ms/step - loss: 0.

In [6]:
# Evaluate
test_error, test_accuracy = model.evaluate(test_dataset, verbose=1)
print("Accuracy on Test Set: ", test_accuracy)
print("Error on Test Set: ", test_error)
model.save("melody_generation_model_complete.keras")

7126/7126 ━━━━━━━━━━━━━━━━━━━━ 68s 9ms/step - loss: 0.5259 - sparse_categorical_accuracy: 0.8436
Accuracy on Test Set:  0.8438318371772766
Error on Test Set:  0.5249736309051514
